In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scapy.all import PcapReader, IP, TCP, UDP, ICMP, DNS
from sklearn.preprocessing import StandardScaler
import pickle
import os
from datetime import datetime
from collections import defaultdict
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')


In [19]:


class BiLSTMInference:
    """Load trained BiLSTM model and perform inference on real traffic"""
    
    def __init__(self, model_path, scaler_path):
        """
        Load pre-trained model and scaler
        
        Args:
            model_path: Path to .h5 model file
            scaler_path: Path to .pkl scaler file
        """
        print("\n" + "="*70)
        print(" LOADING TRAINED MODEL")
        print("="*70)
        
        # Load model
        print(f"\n📦 Loading model from: {model_path}")
        self.model = keras.models.load_model(model_path)
        print(f"✓ Model loaded successfully")
        
        # Load scaler
        print(f"📦 Loading scaler from: {scaler_path}")
        with open(scaler_path, 'rb') as f:
            self.scaler_x = pickle.load(f)
        print(f"✓ Scaler loaded successfully")
        
        # Model info
        print(f"\n📊 Model Information:")
        print(f"   Total parameters: {self.model.count_params():,}")
        
        self.sequence_length = 50
        self.n_features = 13
    
    def extract_sequence_features(self, packet):
        """Extract 13 features from a single packet"""
        features = []
        
        try:
            if IP in packet:
                # Packet length (normalized to 1500 max)
                features.append(min(len(packet) / 1500.0, 1.0))
                
                # IP TTL (normalized to 255 max)
                features.append(packet[IP].ttl / 255.0)
                
                # IP flags
                features.append(int(packet[IP].flags) / 7.0)
                
                # IP ID variation
                features.append((packet[IP].id % 1000) / 1000.0)
                
                if TCP in packet:
                    # TCP ports
                    features.append(packet[TCP].sport / 65535.0)
                    features.append(packet[TCP].dport / 65535.0)
                    
                    # TCP flags
                    features.append(int(packet[TCP].flags) / 63.0)
                    
                    # TCP window size
                    features.append(packet[TCP].window / 65535.0)
                    
                    # Payload length
                    payload_len = len(packet[TCP].payload)
                    features.append(min(payload_len / 1500.0, 1.0))
                    
                    # TCP options count
                    features.append(min(float(len(packet[TCP].options)) / 10.0, 1.0))
                    
                    # Payload entropy
                    features.append(self._payload_entropy(bytes(packet[TCP].payload)))
                    
                    features.extend([0.0, 0.0])
                    
                elif UDP in packet:
                    features.append(packet[UDP].sport / 65535.0)
                    features.append(packet[UDP].dport / 65535.0)
                    
                    payload_len = len(packet[UDP].payload)
                    features.append(min(payload_len / 1500.0, 1.0))
                    
                    features.extend([0.0, 0.0, 0.0, 0.0, 0.0])
                    
                else:
                    features.extend([0.0] * 10)
                
                # ICMP
                features.append(1.0 if ICMP in packet else 0.0)
                
                # DNS
                features.append(1.0 if DNS in packet else 0.0)
            else:
                features = [0.0] * 13
        
        except:
            features = [0.0] * 13
        
        return np.array(features[:13])
    
    def _payload_entropy(self, payload):
        """Calculate normalized Shannon entropy"""
        if not payload or len(payload) == 0:
            return 0.0
        
        from scipy.stats import entropy
        byte_counts = np.bincount(list(payload), minlength=256)
        probabilities = byte_counts[byte_counts > 0] / len(payload)
        ent = entropy(probabilities, base=2) / 8.0
        return min(ent, 1.0)
    
    def extract_flows_from_pcap(self, pcap_file):
        """Extract flows from PCAP file"""
        print(f"\n📂 Reading PCAP: {pcap_file}")
        
        flow_dict = defaultdict(list)
        
        try:
            with PcapReader(pcap_file) as pcap:
                for packet in tqdm(pcap, desc="   Extracting flows"):
                    if IP not in packet:
                        continue
                    
                    # Flow identifier
                    src_ip = packet[IP].src
                    dst_ip = packet[IP].dst
                    proto = packet[IP].proto
                    
                    if TCP in packet:
                        src_port = packet[TCP].sport
                        dst_port = packet[TCP].dport
                    elif UDP in packet:
                        src_port = packet[UDP].sport
                        dst_port = packet[UDP].dport
                    else:
                        src_port = 0
                        dst_port = 0
                    
                    flow_key = tuple(sorted([
                        (src_ip, src_port, proto),
                        (dst_ip, dst_port, proto)
                    ]))
                    
                    # Extract features
                    features = self.extract_sequence_features(packet)
                    flow_dict[flow_key].append(features)
            
            print(f"   ✓ Extracted {len(flow_dict)} flows")
            return flow_dict
        
        except Exception as e:
            print(f"   ❌ Error reading PCAP: {e}")
            return {}
    
    def create_sequences_from_flows(self, flow_dict):
        """Convert flows to sequences"""
        print(f"\n🔄 Creating sequences from flows...")
        
        sequences = []
        flow_labels = []
        
        for flow_key, packets in tqdm(flow_dict.items(), desc="   Converting"):
            if len(packets) >= 3:  # Minimum 3 packets
                # Pad or truncate
                if len(packets) < self.sequence_length:
                    packets = packets + [np.zeros(13)] * (self.sequence_length - len(packets))
                else:
                    packets = packets[:self.sequence_length]
                
                seq = np.array(packets)
                sequences.append(seq)
                flow_labels.append(flow_key)
        
        sequences = np.array(sequences)
        print(f"   ✓ Created {len(sequences)} sequences")
        
        # Normalize
        print(f"   Normalizing sequences...")
        seq_len, n_feat = sequences[0].shape
        X_reshaped = sequences.reshape(-1, n_feat)
        X_scaled = self.scaler_x.transform(X_reshaped)
        sequences = X_scaled.reshape(sequences.shape)
        
        return sequences, flow_labels
    
    def predict_flows(self, sequences, flow_labels, threshold=0.5):
        """Predict steganography on sequences"""
        print(f"\n🔮 Running inference on {len(sequences)} sequences...")
        
        predictions = self.model.predict(sequences, verbose=0)
        pred_labels = (predictions > threshold).astype(int).flatten()
        pred_proba = predictions.flatten()
        
        results = []
        for flow_key, pred, prob in zip(flow_labels, pred_labels, pred_proba):
            result = {
                'flow': f"{flow_key[0][0]}:{flow_key[0][1]} ↔ {flow_key[1][0]}:{flow_key[1][1]}",
                'protocol': flow_key[0][2],
                'prediction': 'STEGANOGRAPHY' if pred == 1 else 'BENIGN',
                'confidence': prob if pred == 1 else (1 - prob),
                'raw_probability': prob
            }
            results.append(result)
        
        return results
    
    def analyze_results(self, results):
        """Analyze and display results"""
        print("\n" + "="*70)
        print(" INFERENCE RESULTS")
        print("="*70)
        
        stego_count = sum(1 for r in results if r['prediction'] == 'STEGANOGRAPHY')
        benign_count = len(results) - stego_count
        
        print(f"\n📊 Summary:")
        print(f"   Total flows analyzed: {len(results)}")
        print(f"   Steganography detected: {stego_count} ({stego_count/len(results)*100:.1f}%)")
        print(f"   Benign flows: {benign_count} ({benign_count/len(results)*100:.1f}%)")
        
        # Sort by confidence
        results_sorted = sorted(results, key=lambda x: x['confidence'], reverse=True)
        
        print(f"\n🔴 HIGH CONFIDENCE STEGANOGRAPHY FLOWS:")
        stego_high = [r for r in results_sorted if r['prediction'] == 'STEGANOGRAPHY' and r['confidence'] > 0.9]
        if stego_high:
            for i, r in enumerate(stego_high[:10], 1):
                print(f"   {i}. {r['flow']}")
                print(f"      Confidence: {r['confidence']:.4f} | Proto: {r['protocol']}")
        else:
            print(f"   None detected")
        
        print(f"\n🟢 HIGH CONFIDENCE BENIGN FLOWS:")
        benign_high = [r for r in results_sorted if r['prediction'] == 'BENIGN' and r['confidence'] > 0.9]
        if benign_high:
            for i, r in enumerate(benign_high[:10], 1):
                print(f"   {i}. {r['flow']}")
                print(f"      Confidence: {r['confidence']:.4f} | Proto: {r['protocol']}")
        else:
            print(f"   None detected")
        
        print(f"\n🟡 UNCERTAIN FLOWS (0.4 < confidence < 0.6):")
        uncertain = [r for r in results_sorted if 0.4 < r['confidence'] < 0.6]
        if uncertain:
            for i, r in enumerate(uncertain[:10], 1):
                print(f"   {i}. {r['flow']}")
                print(f"      Confidence: {r['confidence']:.4f} | Prediction: {r['prediction']}")
        else:
            print(f"   None detected")
        
        return results_sorted
    
    def export_results(self, results, output_csv='inference_results.csv'):
        """Export results to CSV"""
        print(f"\n💾 Exporting results...")
        
        df = pd.DataFrame(results)
        df.to_csv(output_csv, index=False)
        print(f"   ✓ Saved to: {output_csv}")
        
        return df
    
    def visualize_predictions(self, results, output_dir='results'):
        """Visualize prediction distribution"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        print(f"\n📈 Creating visualizations...")
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Prediction distribution
        preds = [r['prediction'] for r in results]
        pred_counts = pd.Series(preds).value_counts()
        
        axes[0, 0].bar(pred_counts.index, pred_counts.values, color=['#2ecc71', '#e74c3c'])
        axes[0, 0].set_title('Prediction Distribution', fontsize=12, fontweight='bold')
        axes[0, 0].set_ylabel('Count')
        for i, v in enumerate(pred_counts.values):
            axes[0, 0].text(i, v + 5, str(v), ha='center', fontweight='bold')
        
        # Confidence distribution
        confidences = [r['confidence'] for r in results]
        axes[0, 1].hist(confidences, bins=30, color='#3498db', edgecolor='black')
        axes[0, 1].set_title('Confidence Distribution', fontsize=12, fontweight='bold')
        axes[0, 1].set_xlabel('Confidence')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].axvline(np.mean(confidences), color='red', linestyle='--', label=f'Mean: {np.mean(confidences):.2f}')
        axes[0, 1].legend()
        
        # Confidence by prediction
        stego_conf = [r['confidence'] for r in results if r['prediction'] == 'STEGANOGRAPHY']
        benign_conf = [r['confidence'] for r in results if r['prediction'] == 'BENIGN']
        
        axes[1, 0].boxplot([stego_conf, benign_conf], labels=['Stego', 'Benign'])
        axes[1, 0].set_title('Confidence by Prediction', fontsize=12, fontweight='bold')
        axes[1, 0].set_ylabel('Confidence')
        axes[1, 0].grid(True, alpha=0.3)
        
        # ROC-like curve
        raw_probs = sorted([r['raw_probability'] for r in results])
        axes[1, 1].plot(raw_probs, color='#9b59b6', linewidth=2)
        axes[1, 1].axhline(0.5, color='red', linestyle='--', label='Decision Boundary')
        axes[1, 1].set_title('Raw Probability Distribution', fontsize=12, fontweight='bold')
        axes[1, 1].set_xlabel('Sample Index (sorted)')
        axes[1, 1].set_ylabel('Probability of Steganography')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plot_path = os.path.join(output_dir, 'inference_visualization.png')
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"   ✓ Saved to: {plot_path}")
        plt.close()



In [ ]:

def main():
    """Main inference pipeline"""
    
    # Model paths (update these with your actual model files)
    model_path = "models/bilstm_stego_20251017_210507.h5"  # Update with your model
    scaler_path = "models/scaler_20251017_210507.pkl"      # Update with your scaler
    
    # Test PCAP
    test_pcap = "stego_dataset\steganography_dataset_20251017_210123.pcap"  # Your test traffic
    
    # Check if model exists
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        print("❌ Model files not found!")
        print(f"   Looking for: {model_path}")
        print(f"   Looking for: {scaler_path}")
        print("\nPlease train a model first using train_bilstm.py")
        return
    
    # Initialize inference
    inference = BiLSTMInference(model_path, scaler_path)
    
    # Extract flows
    flow_dict = inference.extract_flows_from_pcap(test_pcap)
    
    # Create sequences
    sequences, flow_labels = inference.create_sequences_from_flows(flow_dict)
    
    # Run inference
    results = inference.predict_flows(sequences, flow_labels, threshold=0.5)
    
    # Analyze results
    results_sorted = inference.analyze_results(results)
    
    # Export
    df = inference.export_results(results_sorted)
    
    # Visualize
    inference.visualize_predictions(results_sorted)
    
    print("\n" + "="*70)
    print(" ✅ Inference Complete!")
    print("="*70 + "\n")


if __name__ == "__main__":
    main()


 LOADING TRAINED MODEL

📦 Loading model from: models/bilstm_stego_20251017_210507.h5


✓ Model loaded successfully
📦 Loading scaler from: models/scaler_20251017_210507.pkl
✓ Scaler loaded successfully

📊 Model Information:
   Total parameters: 357,505

📂 Reading PCAP: archive/PCAP/RITA pcap files/1.pcap


   Extracting flows: 148895it [01:11, 2069.43it/s]


   ✓ Extracted 957 flows

🔄 Creating sequences from flows...


   Converting: 100%|██████████| 957/957 [00:00<00:00, 75764.91it/s]


   ✓ Created 538 sequences
   Normalizing sequences...

🔮 Running inference on 538 sequences...

 INFERENCE RESULTS

📊 Summary:
   Total flows analyzed: 538
   Steganography detected: 121 (22.5%)
   Benign flows: 417 (77.5%)

🔴 HIGH CONFIDENCE STEGANOGRAPHY FLOWS:
   1. 192.168.43.188:68 ↔ 192.168.43.49:67
      Confidence: 0.9947 | Proto: 17
   2. 142.251.143.129:443 ↔ 192.168.43.188:58914
      Confidence: 0.9939 | Proto: 17
   3. 142.251.143.106:443 ↔ 192.168.43.188:64330
      Confidence: 0.9938 | Proto: 17
   4. 142.250.180.163:443 ↔ 192.168.43.188:61369
      Confidence: 0.9936 | Proto: 17
   5. 142.251.209.42:443 ↔ 192.168.43.188:53115
      Confidence: 0.9935 | Proto: 17
   6. 142.250.180.142:443 ↔ 192.168.43.188:52058
      Confidence: 0.9934 | Proto: 17
   7. 142.251.209.42:443 ↔ 192.168.43.188:63383
      Confidence: 0.9934 | Proto: 17
   8. 142.250.180.142:443 ↔ 192.168.43.188:56596
      Confidence: 0.9934 | Proto: 17
   9. 142.251.143.206:443 ↔ 192.168.43.188:51714
      